<center>
<img src="https://www.iybssd2022.org/wp-content/uploads/ASAQ.jpg" width="150"/> 
</center>

        
<center>
<h1><font color= "blue" size="+2">ASAQ Python Data Analysis Courses</font></h1>
</center>

---

<center><h1><font color="blue" size="+2">Analysis with MovingPandas</font></h1></center>

## <font color="red">Objectives</font>

We want to:

- Provide an overview of MovingPandas and its main data structures.
- Learn how to create and manipulate a MovingPandas trajectory.
- Plot data along a path.

## <font color="red">Required modules/packages</font>

- __Matplotlib__: for basic plots.
- __Pandas__: Manipulation and exploratory data analysis of tabular data.
- __Shapely__: For manipulation and analysis of planar geometric objects
- __GeosPandas__: Combines the capabilities of Pandas and Shapely for geospatial operations
- __MovingPandas__: Handling the movement of geospatial objects.

### <font color="red">Uncomment and run the cell below only if in Google Colab</font>

In [ ]:
#!sudo apt-get update && apt-get install -y libspatialindex-dev
#!pip install rtree
#!pip install geopandas
#!pip install mapclassify
#!pip install movingpandas

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

In [ ]:
import pandas as pd

In [ ]:
import geopandas as gpd

In [ ]:
from shapely import geometry as shpgeom
from shapely import wkt as shpwkt

In [ ]:
import movingpandas as mpd

In [ ]:
import holoviews as hv

In [ ]:
import hvplot.pandas 

In [ ]:
mpd.show_versions()

# <font color="red"> What is MovingPandas? </font>
![fig_logo](https://movingpandas.github.io/movingpandas/assets/img/logo-wide.svg)

- A Python library for handling the movement of geospatial objects.
- Provides trajectory data structures and functions for movement data exploration and analysis.
- It is based based on Pandas, GeoPandas, and HoloViz.

# <font color="red"> Features of MovingPandas</font>

- Create trajectories from diverse sources, including CSV files, GIS file formats, (Geo)DataFrames, and OGC Moving Features JSONs (MF-JSON)
- Find locations for given time stamps and time spans
- Compute movement speed, direction, and sampling intervals
- Detect and extract stops
- Split trajectories into individual trips
- Clean, simplify, generalize, and aggregate trajectories
- Create static and interactive visualizations

# <font color="red"> Understanding Trajectory</font>

A trajectory:
- A time-ordered series of geometries.
   - The geometries and associated attributes are stored in a GeoPandas GeoDataFrame.
- __Can be seen as a sequence of points that specify the position of a moving object in space and time__.
- Can have a parent trajectory and can itself be the parent of successive trajectories.
- Can represent its data either as point-based or as line-based. 
- A segment is a part of the trajectory that contains a list of episodes. 
    - Each episode has a starting and ending timestamp, a segmentation criterion (annotation type), and an episode annotation. 
    - For instance, an annotation type can be the “weather conditions”, and an episode annotation can be “a storm”, “heavy rain”, “extremely high waves”, etc.

## <font color="red">Data Access</font>

File name:

In [ ]:
file_name = "L2-01-08-2023evening.xlsx"
data_url = "https://github.com/JulesKouatchou/asaq_py/raw/main/sample_data/L2-01-08-2023evening.xlsx"
data_url = "/".join(["../sample_data", file_name])

### <font color="blue">Read the file</font>

In [ ]:
df = pd.read_excel(data_url, 
                   sheet_name="Feuil1",
                  parse_dates={'t': [0]}
                  )
df

We can remove the column _time_:

In [ ]:
df = df.drop(columns=['time'])

#### Make the time as the index of the DataFrame

In [ ]:
df.set_index('t', inplace=True)

#### Convert the Pandas DataFrame into a MovingPandas trajectory


```python
Trajectory(df, traj_id, 
           obj_id=None, t=None, x=None, y=None, 
           crs='epsg:4326', parent=None)
```

- __df__ (GeoDataFrame or DataFrame) – GeoDataFrame with point geometry column and timestamp index
- __traj_id__ (any) – Trajectory ID
- __obj_id__ (any) – Moving object ID
- __t__ (string) – Name of the DataFrame column containing the timestamp
- __x__ (string) – Name of the DataFrame column containing the x coordinate
- __y__ (string) – Name of the DataFrame column containing the y coordinate
- __crs__ (string) – CRS of the x/y coordinates
- __parent__ (Trajectory) – Parent trajectory

In [ ]:
traj = mpd.Trajectory(df, x="lng", y="lat", traj_id=1)

In [ ]:
print(traj)

#### Get the trajectory GeoDataFrame

In [ ]:
traj.df

#### Plot the trajectory GeoDataFrame

In [ ]:
traj.df.plot();

#### Extract the individual lines from the trajectory 

In [ ]:
lines = traj.to_line_gdf()
lines

In [ ]:
lines.plot();

#### Compute the sampling interval (median time difference between records):

In [ ]:
traj.get_sampling_interval()

## <font color="blue">Extracting information from the Trajectory</font>

#### Start/End Locations

In [ ]:
str(traj.get_start_location())

In [ ]:
str(traj.get_end_location())

In [ ]:
traj.get_length()

#### Start/End Times

In [ ]:
traj.get_start_time()

In [ ]:
traj.get_end_time()

In [ ]:
traj.get_duration()

## <font color="blue">Processing the Trajectory</font>

- We compute the distance, speed, and acceleration of movement along the trajectory (between consecutive points). 
- The paramters are added as new columns.

#### Distance

In [ ]:
traj.add_distance(overwrite=True, name="distance (m)")

In [ ]:
traj.df

#### Speed

In [ ]:
traj.add_speed(overwrite=True, name="speed (m/s)")

In [ ]:
traj.df

### Visualization

We can turn the trajectory into a linestring:

In [ ]:
traj.to_linestring()

We can directly call the `plot()` function that draws each line segment individually.

In [ ]:
traj.plot();

We can visualize CO2 values where each line segment is colored:

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))

traj.plot(legend=True, 
           column="CO2", 
           capstyle='round', 
              cmap="jet", ax=ax);

We can visualize the speed values where each line segment is colored:

In [ ]:
traj.plot(column="speed (m/s)", linewidth=5, 
                 capstyle='round', legend=True)

## Create an inteactive plot with `hvplot`:

In [ ]:
traj.hvplot(color="blue", line_width=5)

We can select the background image with the by setting the tiles parameter with one of the options:

‘CartoDark’, ‘CartoLight’, ‘EsriImagery’,  ‘OSM’ (default)

In [ ]:
mytile= "EsriImagery"
traj.hvplot(color="blue", line_width=5, tiles=mytile)

#### Plot `CO2` along the trajectory

In [ ]:
fig, ax = plt.subplots(figsize=(12,15))

mytile= "EsriImagery"
traj.hvplot(ax=ax, tiles=mytile, c="CO2", line_width=5, cmap='Dark2')